# **Introduction:**

**Research Question**

This paper endeavors to answer the following question: "How might we create a personalized music recommendation system for users based on their listening history, without being invasive or relying on personal data?"




**Background / Relevance for Study**

 In today's digital age, music streaming services such as Spotify are becoming increasingly popular. However, users often struggle to find new music that suits their tastes with such a vast selection of music. Personalized music recommendation systems have become a popular solution to this problem. Given users' listening history, these systems can suggest new music that they may enjoy. Unfortunately, many existing systems rely heavily on users' personal data (eg: age, location, etc.), which raises concerns about privacy. Our proposed model aims to create a personalized music recommendation system that relies on users' listening history without being invasive. Presently, Spotify recommends content based both on the actual content of songs that a user likes, and also the relationship that one track has with other tracks, determined by a broader set of users. 

Our proposed response to our main query is to to create a 
novel music recommendation algorithm that differs from that of Spotify. Spotify present incorporates multiple recommendation methods, chiefly:


1.   Content-Based Recommendation
2.   Collaborative-Based Recommendation
3.Popularity-based recommendation

It is our objective to create a new method which does not incorporate collaborative-based recommendation. The goal of this change is to enhance the privacy of users such that their listening history is not communicated with other users, directly or indirectly.

As an example illustrating this use case, if a user (Bob) has one friend on spotify (Rob), Bob might be aware that Rob is an avid fan of Norwegian death metal if he receives that as a recommendation. Rob may prefer to keep that private, and would feasibly choose to opt into our algorithm which eschews collaborate-based recommendation in favor of his privacy.


**Varaibles, Parameters, and Assumptions**

Our variables will include users' listening history, the genres and artists of the music they listen to, and their interactions with the music streaming service (such as liking, disliking, or skipping songs). We will assume that users' listening history reflects their music preferences to some extent. We will also assume that the music streaming service has access to a large enough database of music to make relevant recommendations.


**Limitations of Data**

We source our data from KaggleSet data (https://www.kaggle.com/datasets/mrmorj/dataset-of-songs-in-spotify). While KaggleSet provides a large quantity of data, it does not contain every single song on Spotify; thus, not every song on a user's playlist may be represented in the data, renderinig the recommendation algorithm less accurate given its reduced information. In particular, our data source only holds information on: **[Trap, Techno, Techhouse, Trance, Psytrance, Dark Trap, DnB (drums and bass), Hardstyle, Underground Rap, Trap Metal, Emo, Rap, RnB, Pop, Hiphop].** 

Each song has a set of accomanying labels with further data, such as danceability, energy, loudness, musical key, and level of instrumentality (to name a few).

Our project and technical analysis consists of 5 major components, enumerated below:

1. Data Collection and Cleaning: 



## Imports

In [5]:
import pandas as pd
import numpy as np
import json
import re 
import sys
import itertools

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt


import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import spotipy.util as util

import warnings
warnings.filterwarnings("ignore")

In [6]:
%matplotlib inline

In [7]:
#Makes using jupyter notebook on laptops much easier
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

# Data Processing
dataset link: https://www.kaggle.com/datasets/ektanegi/spotifydata-19212020

In [11]:
spotify_df = pd.read_csv('data.csv')

In [12]:
spotify_df.head()

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
0,0.995,['Carl Woitschach'],0.708,158648,0.1950,0,6KbQ3uYMLKb5jDxLF7wYDD,0.563,10,0.1510,-12.428,1,Singende Bataillone 1. Teil,0,1928,0.0506,118.469,0.7790,1928
1,0.994,"['Robert Schumann', 'Vladimir Horowitz']",0.379,282133,0.0135,0,6KuQTIu1KoTTkLXKrwlLPV,0.901,8,0.0763,-28.454,1,"Fantasiestücke, Op. 111: Più tosto lento",0,1928,0.0462,83.972,0.0767,1928
2,0.604,['Seweryn Goszczyński'],0.749,104300,0.2200,0,6L63VW0PibdM1HDSBoqnoM,0.000,5,0.1190,-19.924,0,Chapter 1.18 - Zamek kaniowski,0,1928,0.9290,107.177,0.8800,1928
3,0.995,['Francisco Canaro'],0.781,180760,0.1300,0,6M94FkXd15sOAOQYRnWPN8,0.887,1,0.1110,-14.734,0,Bebamos Juntos - Instrumental (Remasterizado),0,1928-09-25,0.0926,108.003,0.7200,1928
4,0.990,"['Frédéric Chopin', 'Vladimir Horowitz']",0.210,687733,0.2040,0,6N6tiFZ9vLTSOIxkj8qKrd,0.908,11,0.0980,-16.829,1,"Polonaise-Fantaisie in A-Flat Major, Op. 61",1,1928,0.0424,62.149,0.0693,1928
